In [2]:
import os
import pandas as pd


pos_path = ".\\data\\train\\pos\\"
neg_path = ".\\data\\train\\neg\\"

neg_review_list = []
pos_review_list = []
review_list = []
test_set = []

for file in os.listdir(neg_path):
    file_path = os.path.join(neg_path, file)
    fh = open(file_path, 'r', encoding="utf8")
    entry = {'class': 0, 'raw_txt': fh.read()}
    neg_review_list.append(entry)
    fh.close()
    
for file in os.listdir(pos_path):
    file_path = os.path.join(pos_path, file)
    fh = open(file_path, 'r', encoding="utf8")
    entry = {'class': 1, 'raw_txt': fh.read()}
    pos_review_list.append(entry)
    fh.close()

review_list = pd.DataFrame(neg_review_list + pos_review_list)

print("done")

done


In [105]:
import re
from nltk.corpus import stopwords
import numpy as np
from nltk.stem.snowball import SnowballStemmer

stopWords = set(stopwords.words('english'))

def text_processing(txt):
    stemmer = SnowballStemmer("english",ignore_stopwords=True)
    processed = re.sub(r'[^\w\s]','', txt.lower())
    #processed = ' '.join([t for t in processed.split(' ')if t not in stopWords])
    #processed = ' '.join([stemmer.stem(t) for t in processed.split(' ')])
    return processed

def processing(df):
    # lowering and removing punctuation
    df['processed'] = df['raw_txt'].apply(lambda x: text_processing(x))
    # df['processed'] = df['raw_txt'].apply(lambda x: re.sub(r'[^\w\s]','', x.lower()))
                                                                                                                     
    # total length of sentence
    df['length'] = df['processed'].apply(lambda x: len(x))
    # get number of words
    df['words'] = df['processed'].apply(lambda x: len(x.split(' ')))
    df['words_not_stopword'] = df['processed'].apply(lambda x: len([t for t in x.split(' ')if t not in stopWords]))
    # get the average word length
    df['avg_word_length'] = df['processed'].apply(
        lambda x: np.mean([len(t) for t in x.split(' ') if t not in stopWords]) if len(
            [len(t) for t in x.split(' ') if t not in stopWords]) > 0 else 0)
    # get the average word length
    df['commas'] = df['raw_txt'].apply(lambda x: x.count(','))

    return df


df = processing(review_list)
print(df.head())



   class                                            raw_txt  \
0      0  Story of a man who has unnatural feelings for ...   
1      0  Airport '77 starts as a brand new luxury 747 p...   
2      0  This film lacked something I couldn't put my f...   
3      0  Sorry everyone,,, I know this is supposed to b...   
4      0  When I was little my parents took me along to ...   

                                           processed  \
0  story of a man who has unnatural feelings for ...   
1  airport 77 starts as a brand new luxury 747 pl...   
2  this film lacked something i couldnt put my fi...   
3  sorry everyone i know this is supposed to be a...   
4  when i was little my parents took me along to ...   

                                   text_not_stopword  length  words  \
0  stori man unnatur feel pig start open scene te...     644    112   
1  airport 77 start brand new luxuri 747 plane lo...    4324    801   
2  film lack someth couldnt put finger first char...     776    141   


In [78]:
from sklearn.model_selection import train_test_split

features = [f for f in df.columns.values if f not in ['class']]
x_train, x_test, y_train, y_test = train_test_split(df[features], df['class'],
                                                    test_size=0.20, random_state=10)
x_set = df[features]
target_set = df['class']
print(x_train.head())

                                                 raw_txt  \
5610   Devil Hunter gained notoriety for the fact tha...   
3742   What can I say about Seven Pounds...well I wat...   
5692   This film was so predictable, that during the ...   
22213  In an attempt to cash in on the success of Uni...   
23165  Dark comedy? Gallows humor? How does one make ...   

                                               processed  \
5610   devil hunter gained notoriety for the fact tha...   
3742   what can i say about seven poundswell i watche...   
5692   this film was so predictable that during the e...   
22213  in an attempt to cash in on the success of uni...   
23165  dark comedy gallows humor how does one make a ...   

                                       text_not_stopword  length  words  \
5610   devil hunter gained notoriety fact dpp video n...    1405    280   
3742   say seven poundswell watched flight seattle to...     727    133   
5692   film predictable entire time youre hoping obvi

In [6]:
from sklearn.base import BaseEstimator, TransformerMixin


class TxtPicker(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]


class NumberSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]


In [98]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer,HashingVectorizer
from sklearn.preprocessing import StandardScaler


tf_id_feature = Pipeline([
    ('selector', TxtPicker(key='processed')),            
    ('tfidf', TfidfVectorizer())
])

binary_count_feature = Pipeline([
    ('selector', TxtPicker(key='processed')),            
    ('count', HashingVectorizer(ngram_range=(1, 1)))
])

length = Pipeline([
    ('selector', NumberSelector(key='length')),
    ('standard', StandardScaler())
])

words = Pipeline([
    ('selector', NumberSelector(key='words')),
    ('standard', StandardScaler())
])


In [ ]:

from sklearn.pipeline import FeatureUnion
feats_tfid = FeatureUnion([('text', tf_id_feature), 
                      ('length', length),
                      ('words', words)
                      ])

feats_count = FeatureUnion([('text', binary_count_feature), 
])

In [113]:

from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn import tree


svc_pipeline = Pipeline([
    ('feats', feats_tfid),
    ('selector',SelectFromModel(LogisticRegression(solver='lbfgs'))),
    ('classifier', LinearSVC())])

logistic_pipeline = Pipeline([
    ('feats', feats_tfid),
    ('selector',SelectFromModel(LinearSVC())),
    ('classifier', LogisticRegression())])

decision_tree_pipeline = Pipeline([
    ('feats', feats_tfid),
    ('classifier', tree.DecisionTreeClassifier())])

svc_pipeline.fit(x_train, y_train)
preds = svc_pipeline.predict(x_test)

np.mean(preds == y_test)


C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-p

C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\linear_model\logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


0.8904

In [88]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(pipeline, x_set, target_set, cv=3)

C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-p

C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-p

C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-p

C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [90]:
print("Accuracy: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.860 (+/- 0.01)


In [63]:
test_path = ".\\data\\test\\"
test_set = []

numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

dir_list = sorted(os.listdir(test_path), key=numericalSort)

for file in dir_list:
    file_path = os.path.join(test_path, file)
    fh = open(file_path, 'r', encoding="utf8")
    entry = {'raw_txt': fh.read()}
    test_set.append(entry)
    fh.close()

test_set = pd.DataFrame(test_set)
print(test_set.head())
print(test_set.shape)


                                             raw_txt
0  Thinking that it could only get better was the...
1  For most people, RoboCop 3 is the film that re...
2  I'm pretty sure Poe would have considered this...
3  This is one of those made-for-TV B movies that...
4  Wallace & Gromit have been around for some tim...
(25000, 1)


In [65]:

test_set = processing(test_set)

test_predictions = pipeline.predict(test_set)

prediction_df = pd.DataFrame(test_predictions)

prediction_df.columns = ['Category']
print(test_set.head())
print(prediction_df.head())

C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\HollowTube\Anaconda3\envs\MiniProject1\lib\site-packages\sklearn\pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


                                             raw_txt  \
0  Thinking that it could only get better was the...   
1  For most people, RoboCop 3 is the film that re...   
2  I'm pretty sure Poe would have considered this...   
3  This is one of those made-for-TV B movies that...   
4  Wallace & Gromit have been around for some tim...   

                                           processed  \
0  thinking that it could only get better was the...   
1  for most people robocop 3 is the film that rea...   
2  im pretty sure poe would have considered this ...   
3  this is one of those madefortv b movies that i...   
4  wallace  gromit have been around for some time...   

                                   text_not_stopword  length  words  \
0  thinking could get better worst assumption eve...     541    106   
1  people robocop 3 film really big disgrace robo...    5125    998   
2  im pretty sure poe would considered travesty f...     336     58   
3  one madefortv b movies awful kind endea

In [66]:
export_csv = prediction_df.to_csv (r'.\test_results.csv', header=True)
